In [73]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
import warnings
warnings.filterwarnings('ignore')


## Get Data & load as DataFrame

*curl* is a Bash command. You can execute Bash commands in a Jupyter notebook by prefixing them with an exclamation mark. This command downloads a CSV file from Azure blob storage and saves it using the name flightdata.csv.
*Importing the dataset*

In [2]:
!curl https://topcs.blob.core.windows.net/public/FlightData.csv -o flightdata.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1552k  100 1552k    0     0  2208k      0 --:--:-- --:--:-- --:--:-- 2205k


In [52]:

# Loading the dataset
df = pd.read_csv('flightdata.csv')
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,...,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,Unnamed: 25
0,2016,1,1,1,5,DL,N836DN,1399,10397,ATL,...,2143,2102.0,-41.0,0.0,0.0,0.0,338.0,295.0,2182.0,NaN
1,2016,1,1,1,5,DL,N964DN,1476,11433,DTW,...,1435,1439.0,4.0,0.0,0.0,0.0,110.0,115.0,528.0,NaN
2,2016,1,1,1,5,DL,N813DN,1597,10397,ATL,...,1215,1142.0,-33.0,0.0,0.0,0.0,335.0,300.0,2182.0,NaN
3,2016,1,1,1,5,DL,N587NW,1768,14747,SEA,...,1335,1345.0,10.0,0.0,0.0,0.0,196.0,205.0,1399.0,NaN
4,2016,1,1,1,5,DL,N836DN,1823,14747,SEA,...,607,615.0,8.0,0.0,0.0,0.0,247.0,259.0,1927.0,NaN


In [ ]:
# df.to_csv('dataset.csv', sep="|", index=False)

| Column| Description|
| --- | --- |
| YEAR | Year the flight took place |

Source: Click for more [Docs](https://docs.microsoft.com/en-us/learn/modules/predict-flight-delays-with-python/2-clean-and-prepare-data)

### Note:
The dataset includes a roughly even distribution of dates throughout the year, which is important because a flight out of Minneapolis is less likely to be delayed due to winter storms in July than it is in January. But this dataset is far from being "clean" and ready to use. Let's clean it up.

One of the most important aspects of preparing a dataset for use in machine learning is selecting the "feature" columns that are relevant to the outcome you are trying to predict while filtering out columns that do not affect the outcome, could bias it in a negative way, or might produce **[multicollinearity](https://en.wikipedia.org/wiki/Multicollinearity)**. Another important task is to eliminate missing values, either by deleting the rows or columns containing them or replacing them with meaningful values. Next is to eliminate extraneous columns and replace missing values in the remaining columns.

## Exploratory Data Analysis/Data Prep

In [53]:
# DataFrame contains {11,231} rows and {26} columns

df.shape

(11231, 26)

In [54]:
# check for missing values

df.isnull().values.any()

True

In [55]:
#  find out where the missing values are

df.isnull().sum()

YEAR                       0
QUARTER                    0
MONTH                      0
DAY_OF_MONTH               0
DAY_OF_WEEK                0
UNIQUE_CARRIER             0
TAIL_NUM                   0
FL_NUM                     0
ORIGIN_AIRPORT_ID          0
ORIGIN                     0
DEST_AIRPORT_ID            0
DEST                       0
CRS_DEP_TIME               0
DEP_TIME                 107
DEP_DELAY                107
DEP_DEL15                107
CRS_ARR_TIME               0
ARR_TIME                 115
ARR_DELAY                188
ARR_DEL15                188
CANCELLED                  0
DIVERTED                   0
CRS_ELAPSED_TIME           0
ACTUAL_ELAPSED_TIME      188
DISTANCE                   0
Unnamed: 25            11231
dtype: int64

Curiously, the 26th column ("Unnamed: 25") contains 11,231 missing values, which equals the number of rows in the dataset. This column was mistakenly created because the CSV file imported contains a comma at the end of each line. **Eliminate that column:**

In [56]:
df = df.drop('Unnamed: 25', axis=1)
df.isnull().sum()

# df.isnull().sum()

YEAR                     0
QUARTER                  0
MONTH                    0
DAY_OF_MONTH             0
DAY_OF_WEEK              0
UNIQUE_CARRIER           0
TAIL_NUM                 0
FL_NUM                   0
ORIGIN_AIRPORT_ID        0
ORIGIN                   0
DEST_AIRPORT_ID          0
DEST                     0
CRS_DEP_TIME             0
DEP_TIME               107
DEP_DELAY              107
DEP_DEL15              107
CRS_ARR_TIME             0
ARR_TIME               115
ARR_DELAY              188
ARR_DEL15              188
CANCELLED                0
DIVERTED                 0
CRS_ELAPSED_TIME         0
ACTUAL_ELAPSED_TIME    188
DISTANCE                 0
dtype: int64

Filter the dataset to eliminate columns that aren't relevant to a predictive model. For example, the aircraft's tail number probably has little bearing on whether a flight will arrive on time, and at the time you book a ticket, you have no way of knowing whether a flight will be cancelled, diverted, or delayed. By contrast, the scheduled departure time could have a lot to do with on-time arrivals. Because of the hub-and-spoke system used by most airlines, morning flights tend to be on time more often than afternoon or evening flights. And at some major airports, traffic stacks up during the day, increasing the likelihood that later flights will be delayed.

In [57]:
df_trans = df[["MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK", "ORIGIN", "DEST", "CRS_DEP_TIME", "ARR_DEL15"]]
df_trans.isnull().sum()

MONTH             0
DAY_OF_MONTH      0
DAY_OF_WEEK       0
ORIGIN            0
DEST              0
CRS_DEP_TIME      0
ARR_DEL15       188
dtype: int64

In [9]:

# dummy variable to experiment filters in PBI 
# df_trans = df[["MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK", "ORIGIN", "DEST", "CRS_DEP_TIME", "ARR_DEL15"]]



In [10]:
# df_trans[['Origination','Destination']] = df[['ORIGIN','DEST']]

In [58]:
df_trans.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,DEST,CRS_DEP_TIME,ARR_DEL15
0,1,1,5,ATL,SEA,1905,0.0
1,1,1,5,DTW,MSP,1345,0.0
2,1,1,5,ATL,SEA,940,0.0
3,1,1,5,SEA,MSP,819,0.0
4,1,1,5,SEA,DTW,2300,0.0


In [59]:
# df_trans.drop('Originated', axis=1).head()
df_trans[df_trans.isnull().values.any(axis=1)].head()


,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,DEST,CRS_DEP_TIME,ARR_DEL15
177,1,9,6,MSP,SEA,701,NaN
179,1,10,7,MSP,DTW,1348,NaN
184,1,10,7,MSP,DTW,625,NaN
210,1,10,7,DTW,MSP,1200,NaN
478,1,22,5,SEA,JFK,2305,NaN


The only column that now contains missing values is the ARR_DEL15 column, which uses 0s to identify flights that arrived on time and 1s for flights that didn't. Show the first five rows with missing values:

In [64]:
# df[df.isnull().values.any(axis=1)].head()
dfcols = df_trans[['ORIGIN','DEST']]

The reason these rows are missing ARR_DEL15 values is that they all correspond to flights that were canceled or diverted. You could call dropna on the DataFrame to remove these rows. But since a flight that is canceled or diverted to another airport could be considered "late," use the fillna method to replace the missing values with 1s.

In [61]:
df = df.fillna({'ARR_DEL15': 1})
df.iloc[177:185] # confirm the Nans in these rows {177-184} were replaced 

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,...,DEP_DEL15,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE
177,2016,1,1,9,6,DL,N3743H,2834,13487,MSP,...,0.0,852,1151.0,NaN,1.0,0.0,1.0,231.0,NaN,1399.0
178,2016,1,1,9,6,DL,N975AT,2839,11433,DTW,...,0.0,1724,1709.0,-15.0,0.0,0.0,0.0,117.0,106.0,509.0
179,2016,1,1,10,7,DL,N924DN,86,13487,MSP,...,NaN,1632,NaN,NaN,1.0,1.0,0.0,104.0,NaN,528.0
180,2016,1,1,10,7,DL,N671DN,87,11433,DTW,...,1.0,1649,1703.0,14.0,0.0,0.0,0.0,129.0,119.0,528.0
181,2016,1,1,10,7,DL,N319NB,423,12478,JFK,...,0.0,1600,1607.0,7.0,0.0,0.0,0.0,155.0,151.0,760.0
182,2016,1,1,10,7,DL,N587NW,440,12478,JFK,...,0.0,849,835.0,-14.0,0.0,0.0,0.0,159.0,146.0,760.0
183,2016,1,1,10,7,DL,N813DN,485,12478,JFK,...,1.0,1945,1955.0,10.0,0.0,0.0,0.0,390.0,335.0,2422.0
184,2016,1,1,10,7,DL,N922DX,557,13487,MSP,...,0.0,912,1500.0,NaN,1.0,0.0,1.0,107.0,NaN,528.0


In [60]:
df_trans = df_trans.fillna({'ARR_DEL15': 1})
df_trans.iloc[177:180] # confirm the Nans in these rows {177-184} were replaced 

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,DEST,CRS_DEP_TIME,ARR_DEL15
177,1,9,6,MSP,SEA,701,1.0
178,1,9,6,DTW,JFK,1527,0.0
179,1,10,7,MSP,DTW,1348,1.0


The dataset is now "clean" in the sense that missing values have been replaced and the list of columns has been narrowed to those most relevant to the model. But there is more to do to prepare the dataset for use in machine learning.

In [62]:
df_trans.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,DEST,CRS_DEP_TIME,ARR_DEL15
0,1,1,5,ATL,SEA,1905,0.0
1,1,1,5,DTW,MSP,1345,0.0
2,1,1,5,ATL,SEA,940,0.0
3,1,1,5,SEA,MSP,819,0.0
4,1,1,5,SEA,DTW,2300,0.0


The CRS_DEP_TIME column of the dataset represents scheduled departure times. The granularity of the numbers in this column — it contains more than 500 unique values — could have a negative impact on accuracy in a machine-learning model. This can be resolved using a technique called [binning](https://towardsdatascience.com/binning-for-feature-engineering-in-machine-learning-d3b3d76f364a) or quantization. What if you divided each number in this column by 100 and rounded down to the nearest integer? 1030 would become 10, 1925 would become 19, and so on, and would be left with a maximum of 24 discrete values in this column. Intuitively, it makes sense, because it probably doesn't matter much whether a flight leaves at 10:30 a.m. or 10:40 a.m. It matters a great deal whether it leaves at 10:30 a.m. or 5:30 p.m. <br>

In [63]:
import math

for index, row in df.iterrows():
    df_trans.loc[index, 'CRS_DEP_TIME'] = math.floor(row['CRS_DEP_TIME'] / 100)
df_trans.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,DEST,CRS_DEP_TIME,ARR_DEL15
0,1,1,5,ATL,SEA,19,0.0
1,1,1,5,DTW,MSP,13,0.0
2,1,1,5,ATL,SEA,9,0.0
3,1,1,5,SEA,MSP,8,0.0
4,1,1,5,SEA,DTW,23,0.0


In addition, the dataset's ORIGIN and DEST columns contain airport codes that represent categorical machine-learning values. These columns need to be converted into discrete columns containing indicator variables, sometimes known as "dummy" variables. In other words, the ORIGIN column, which contains five airport codes, needs to be converted into five columns, one per airport, with each column containing 1s and 0s indicating whether a flight originated at the airport that the column represents. The DEST column needs to be handled in a similar manner.<br>

"Bin" the departure times in the CRS_DEP_TIME column and use Pandas' [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) method to create indicator columns from the ORIGIN and DEST columns.

In [47]:
# df = pd.get_dummies(df, columns=['ORIGIN', 'DEST'])
# df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,...,DEP_DEL15,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE
0,2016,1,1,1,5,DL,N836DN,1399,10397,ATL,...,0.0,2143,2102.0,-41.0,0.0,0.0,0.0,338.0,295.0,2182.0
1,2016,1,1,1,5,DL,N964DN,1476,11433,DTW,...,0.0,1435,1439.0,4.0,0.0,0.0,0.0,110.0,115.0,528.0
2,2016,1,1,1,5,DL,N813DN,1597,10397,ATL,...,0.0,1215,1142.0,-33.0,0.0,0.0,0.0,335.0,300.0,2182.0
3,2016,1,1,1,5,DL,N587NW,1768,14747,SEA,...,0.0,1335,1345.0,10.0,0.0,0.0,0.0,196.0,205.0,1399.0
4,2016,1,1,1,5,DL,N836DN,1823,14747,SEA,...,0.0,607,615.0,8.0,0.0,0.0,0.0,247.0,259.0,1927.0


In [65]:
dummies = pd.get_dummies(df_trans, columns=["ORIGIN", "DEST"])
df_ml = pd.concat([dfcols, dummies], axis=1)
df_ml.head()


,ORIGIN,DEST,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,CRS_DEP_TIME,ARR_DEL15,ORIGIN_ATL,ORIGIN_DTW,ORIGIN_JFK,ORIGIN_MSP,ORIGIN_SEA,DEST_ATL,DEST_DTW,DEST_JFK,DEST_MSP,DEST_SEA
0,ATL,SEA,1,1,5,19,0.0,1,0,0,0,0,0,0,0,0,1
1,DTW,MSP,1,1,5,13,0.0,0,1,0,0,0,0,0,0,1,0
2,ATL,SEA,1,1,5,9,0.0,1,0,0,0,0,0,0,0,0,1
3,SEA,MSP,1,1,5,8,0.0,0,0,0,0,1,0,0,0,1,0
4,SEA,DTW,1,1,5,23,0.0,0,0,0,0,1,0,1,0,0,0


## Prepare Prediction Model

To create a machine learning model, you need two datasets: one for training and one for testing. In practice, you often have only one dataset, so you split it into two.<br> Perform an 80-20 split on the DataFrame you prepared in the previous lab so you can use it to train a machine learning model. You will also separate the DataFrame into feature columns and label columns. The former contains the columns used as input to the model (for example, the flight's origin and destination and the scheduled departure time), while the latter contains the column that the model will attempt to predict — in this case, the ARR_DEL15 column, which indicates whether a flight will arrive on time.

In [74]:

train_x, test_x, train_y, test_y = train_test_split(df_ml.drop('ARR_DEL15', axis=1), df_ml['ARR_DEL15'], test_size=0.2, random_state=42)

In [75]:
train_x.shape

(8984, 16)

In [76]:
test_x.shape

(2247, 16)

In [77]:
train_y.shape

(8984,)

In [71]:
test_y.shape

(2247,)

One of the benefits of using scikit-learn is that you don't have to build these models — or implement the algorithms that they use — by hand. Scikit-learn includes a variety of classes for implementing common machine learning models. One of them is [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), which fits multiple decision trees to the data and uses averaging to boost the overall accuracy and limit [overfitting](https://en.wikipedia.org/wiki/Overfitting)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=13)
model.fit(train_x, train_y)


In [ ]:
predicted = model.predict(test_x)
model.score(test_x, test_y)

In [ ]:
prob = model.predict_proba(test_x)[:,1]
prob

In [ ]:
print("Using RFC, Number of mislabeled points out of a total %d points : %d"
       % (test_x.shape[0], (test_y != predicted).sum()))

In [ ]:


gnb = GaussianNB()


In [ ]:
y_pred = gnb.fit(train_x, train_y).predict(test_x)
print("Using Naive Bayes, Number of mislabeled points out of a total %d points : %d"
       % (test_x.shape[0], (test_y != y_pred).sum()))
      

In [ ]:
gnb.fit(train_x, train_y).predict(test_x)
gnb.score(test_x, test_y)

In [ ]:
from sklearn.metrics import roc_auc_score

probabilities = model.predict_proba(test_x)
roc_auc_score(test_y, probabilities[:, 1])

In [ ]:
probs = gnb.predict_proba(test_x)
roc_auc_score(test_y, probs[:, 1])

# type(y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_y, predicted)

In [ ]:
confusion_matrix(test_y, y_pred)

In [ ]:
from sklearn.metrics import precision_score

train_predictions = model.predict(train_x)
precision_score(train_y, train_predictions)

In [ ]:
train_prediction = gnb.predict(train_x)
precision_score(train_y, train_prediction)

In [ ]:
from sklearn.metrics import recall_score

recall_score(train_y, train_predictions)

In [ ]:
recall_score(train_y, train_prediction)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve
sns.set()

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(test_y, probabilities[:, 1])
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], color='grey', lw=1, linestyle='--')
plt.xlabel('False Positive Rate \n Random Forest Classifier')
plt.ylabel('True Positive Rate')

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(test_y, probs[:, 1])
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], color='grey', lw=1, linestyle='--')
plt.xlabel('False Positive Rate \n Naive Bayes')
plt.ylabel('True Positive Rate')

In [ ]:

# !conda install -c anaconda seaborn -y

In [ ]:
def predict_delay(departure_date_time, origin, destination):
    from datetime import datetime

    try:
        departure_date_time_parsed = datetime.strptime(departure_date_time, '%d/%m/%Y %H:%M:%S')
    except ValueError as e:
        return 'Error parsing date/time - {}'.format(e)

    month = departure_date_time_parsed.month
    day = departure_date_time_parsed.day
    day_of_week = departure_date_time_parsed.isoweekday()
    hour = departure_date_time_parsed.hour

    origin = origin.upper()
    destination = destination.upper()

    input = [{'MONTH': month,
              'DAY': day,
              'DAY_OF_WEEK': day_of_week,
              'CRS_DEP_TIME': hour,
              'ORIGIN_ATL': 1 if origin == 'ATL' else 0,
              'ORIGIN_DTW': 1 if origin == 'DTW' else 0,
              'ORIGIN_JFK': 1 if origin == 'JFK' else 0,
              'ORIGIN_MSP': 1 if origin == 'MSP' else 0,
              'ORIGIN_SEA': 1 if origin == 'SEA' else 0,
              'DEST_ATL': 1 if destination == 'ATL' else 0,
              'DEST_DTW': 1 if destination == 'DTW' else 0,
              'DEST_JFK': 1 if destination == 'JFK' else 0,
              'DEST_MSP': 1 if destination == 'MSP' else 0,
              'DEST_SEA': 1 if destination == 'SEA' else 0 }]

    return model.predict_proba(pd.DataFrame(input))[0][0]

In [ ]:
predict_delay('1/10/2018 21:45:00', 'JFK', 'ATL')

In [ ]:
predict_delay('2/10/2018 21:45:00', 'JFK', 'ATL')

In [ ]:
predict_delay('2/10/2018 10:00:00', 'ATL', 'SEA')

In [ ]:
from sklearn.externals import joblib

# Save the model as a pickle in a file
joblib.dump(model, 'onflight-model.pkl')

In [ ]:
import numpy as np

labels = ('Oct 1', 'Oct 2', 'Oct 3', 'Oct 4', 'Oct 5', 'Oct 6', 'Oct 7')
values = (predict_delay('1/10/2018 21:45:00', 'JFK', 'ATL'),
          predict_delay('2/10/2018 21:45:00', 'JFK', 'ATL'),
          predict_delay('3/10/2018 21:45:00', 'JFK', 'ATL'),
          predict_delay('4/10/2018 21:45:00', 'JFK', 'ATL'),
          predict_delay('5/10/2018 21:45:00', 'JFK', 'ATL'),
          predict_delay('6/10/2018 21:45:00', 'JFK', 'ATL'),
          predict_delay('7/10/2018 21:45:00', 'JFK', 'ATL'))
alabels = np.arange(len(labels))

plt.bar(alabels, values, align='center', alpha=0.5)
plt.xticks(alabels, labels)
plt.ylabel('Probability of On-Time Arrival')
plt.ylim((0.0, 1.0))

In [ ]:
pwd